In [1]:
from datetime import datetime, timedelta, timezone
import pandas as pd
from robo_trader import Trader
from robo_trader.strategies import LorentzianStrategy, LorentzianSettings, LorentzianOptimizer, LorentzianOptimizerSettings
from robo_trader.feeds import YFinanceFeed
import json
import matplotlib.pyplot as plt



In [2]:
# SYMBOL = "BTC-GBP"
# inception_date = datetime(2000, 1, 1, tzinfo=timezone.utc)
# train_since = datetime(2016, 7, 1, tzinfo=timezone.utc)
# start_date = datetime(2018, 7, 1, tzinfo=timezone.utc)
# end_date = datetime(2019, 7, 1, tzinfo=timezone.utc)
# interval = Interval.ONE_DAY
# period = 28

SYMBOL = "BTC-GBP"
inception_date = datetime(2000, 1, 1, tzinfo=timezone.utc)
train_since = datetime(2024, 3, 11, tzinfo=timezone.utc)
start_date = datetime(2024, 4, 11, tzinfo=timezone.utc)
end_date = datetime(2024, 5, 11, tzinfo=timezone.utc)
period = 24

feed = YFinanceFeed(interval = '1h')

In [3]:
default_settings = LorentzianSettings(
    use_RSI=True,
    use_WT=True,
    use_CCI=True,
    use_ADX=False,
    use_MFI=True,
    useKernelSmoothing=True,
    useRegimeFilter=True,
    useAdxFilter=False,
    useVolatilityFilter=False,
    useEmaFilter=False,
    useSmaFilter=False,
    useDynamicExits=True,

    neighborsCount=8, maxBarsBack=720,
    emaPeriod=200, smaPeriod=200,
    lookbackWindow=16, relativeWeight=8.0, regressionLevel=18, crossoverLag=0,
    regimeThreshold=1, adxThreshold=20,
    RSI_param1=24, RSI_param2=5,
    WT_param1=6, WT_param2=11,
    CCI_param1=24, CCI_param2=6,
    ADX_param1=21, ADX_param2=2,
    MFI_param1=14,

    position_size = 0.25,
    trailing_stop = 0.02
)

optimizer_settings = LorentzianOptimizerSettings(
)

settings = LorentzianOptimizer.get_optimal_settings(
    SYMBOL, 1000, feed, optimizer_settings, default_settings, start_date, end_date, period, train_since
)

# Write settings to disk
settings_file_path = "optimal_settings.json"
with open(settings_file_path, "w") as settings_file:
    settings_dict = settings.__dict__
    json.dump(settings_dict, settings_file, indent=4)
    
print(f"Optimal settings have been written to {settings_file_path}")

Optimal settings have been written to optimal_settings.json


In [4]:
with open(settings_file_path, "r") as settings_file:
    optimal_settings_data = json.load(settings_file)
    optimal_settings = LorentzianSettings(**optimal_settings_data)

strategy = LorentzianStrategy(default_settings)

# Create a Trader instance with the strategy
trader = Trader(SYMBOL, 1000, feed, strategy)

# Run the backtest
results = trader.backtest(start_date, end_date, period, train_since, transaction_cost=0.004, trailing_stop_penalty_relief=0.9)


Request to open long position on BTC-GBP
    Opening long position for BTC-GBP with for 25.0% of the cash (£250.0) and 2.0% trail
    Creating MARKET BUY order for BTC-GBP at 52612.2890625 with quantity 0.004732734584199598, trail 0.02, limit None, stop None
    Executing MARKET BUY order for BTC-GBP at 52612.2890625 with quantity 0.004732734584199598, total cost £249.996 and transaction cost £0.996
    Bought 0.004732734584199598 BTC-GBP at 52612.2890625
    Creating TRAILING_STOP SELL order for BTC-GBP at 52612.2890625 with quantity 0.004732734584199598, trail 0.02, limit None, stop 51560.04328125
    Updated trailing stop to 52271.01796875
    Updated trailing stop to 52347.224453124996
    Executing TRAILING_STOP SELL order for BTC-GBP at 52347.224453124996 with quantity 0.004732734584199598, total cost £248.73650163438822 and transaction cost £0.9909820782246543
    Sold 0.004732734584199598 BTC-GBP at 52347.224453124996
Request to open long position on BTC-GBP
    Opening long po

In [5]:
# Set display options
pd.set_option('display.max_rows', None)  # Adjust None to your specific needs
pd.set_option('display.max_columns', None)  # Adjust None as needed
pd.set_option('display.width', 1000)  # Adjust the width to fit your notebook

display(results['returns'])

# Display the results
# Filter the history DataFrame to show only rows with True in open_long or close_long columns
filtered_history = results['history'][(results['history']['open_long'] == True) | (results['history']['close_long'] == True)]

# Display the filtered results
display(filtered_history)

display(results['orders'])


,start_date,end_date,price_start,price_end,valuation_start,valuation_end,asset_return,portfolio_return
0,2024-04-11 00:00:00+00:00,2024-04-11 00:00:00+00:00,56343.437500,56244.683594,1000.000000,1000.000000,-0.001753,0.000000
1,2024-04-11 00:00:00+00:00,2024-04-12 00:00:00+00:00,56244.683594,56004.929688,1000.000000,1000.000000,-0.004263,0.000000
2,2024-04-12 00:00:00+00:00,2024-04-13 00:00:00+00:00,56004.929688,53707.167969,1000.000000,1000.000000,-0.041028,0.000000
3,2024-04-13 00:00:00+00:00,2024-04-14 00:00:00+00:00,53707.167969,51107.062500,1000.000000,1000.000000,-0.048413,0.000000
4,2024-04-14 00:00:00+00:00,2024-04-15 00:00:00+00:00,51107.062500,52612.289062,1000.000000,999.004000,0.029452,-0.000996
5,2024-04-15 00:00:00+00:00,2024-04-16 00:00:00+00:00,52612.289062,50873.664062,999.004000,996.758537,-0.033046,-0.002248
6,2024-04-16 00:00:00+00:00,2024-04-17 00:00:00+00:00,50873.664062,51509.640625,996.758537,996.758537,0.012501,0.000000
7,2024-04-17 00:00:00+00:00,2024-04-18 00:00:00+00:00,51509.640625,49380.281250,996.758537,996.758537,-0.041339,0.000000
8,2024-04-18 00:00:00+00:00,2024-04-19 00:00:00+00:00,49380.281250,50533.406250,996.758537,997.010258,0.023352,0.000253
9,2024-04-19 00:00:00+00:00,2024-04-20 00:00:00+00:00,50533.406250,51407.269531,997.010258,994.767008,0.017293,-0.002250


,open,high,low,close,volume,Dividends,Stock Splits,open_long,close_long
date,,,,,,,,,
2024-04-15 00:00:00+00:00,52775.769531,52885.183594,52485.019531,52612.289062,0,0.0,0.0,True,False
2024-04-15 14:00:00+00:00,52878.800781,52954.550781,51602.589844,51602.589844,32843776,0.0,0.0,False,True
2024-04-18 11:00:00+00:00,49587.710938,50540.894531,49587.710938,50281.285156,1465223168,0.0,0.0,True,False
2024-04-19 01:00:00+00:00,50482.281250,50491.027344,49113.312500,49218.484375,1770577920,0.0,0.0,False,True
2024-04-28 03:00:00+00:00,51215.609375,51477.652344,51195.480469,51251.640625,408448000,0.0,0.0,True,False
2024-04-28 11:00:00+00:00,50380.253906,50380.253906,50145.652344,50214.554688,85643264,0.0,0.0,False,True
2024-04-29 22:00:00+00:00,50195.722656,50856.496094,50195.722656,50845.214844,0,0.0,0.0,True,False
2024-04-30 07:00:00+00:00,50587.351562,50600.097656,50347.085938,50347.085938,0,0.0,0.0,False,True
2024-05-07 09:00:00+00:00,51254.921875,51302.234375,51132.746094,51151.695312,0,0.0,0.0,True,False


{'b2702536-ac46-4ee8-bdee-bd1a557165c2': Order(symbol='BTC-GBP', order_type='MARKET', order_side='BUY', quantity=0.004732734584199598, trail=0.02, limit=None, stop=None, id='b2702536-ac46-4ee8-bdee-bd1a557165c2', timestamp=Timestamp('2024-04-15 00:00:00+0000', tz='UTC'), status='FILLED', trades=[Trade(id='1ef7dd21-1e39-4a9a-bca9-ba6462739e58', order_id='b2702536-ac46-4ee8-bdee-bd1a557165c2', symbol='BTC-GBP', order_type='MARKET', order_side='BUY', price=52612.2890625, quantity=0.004732734584199598, timestamp=Timestamp('2024-04-15 00:00:00+0000', tz='UTC'), transaction_costs=0.996)]),
 '96c2a7be-4481-49a4-bfda-8176e13b1627': Order(symbol='BTC-GBP', order_type='TRAILING_STOP', order_side='SELL', quantity=0.004732734584199598, trail=0.02, limit=None, stop=52347.224453124996, id='96c2a7be-4481-49a4-bfda-8176e13b1627', timestamp=Timestamp('2024-04-15 00:00:00+0000', tz='UTC'), status='FILLED', trades=[Trade(id='1e5d34f3-b1aa-4394-843e-6ea7491d036d', order_id='96c2a7be-4481-49a4-bfda-8176e13